In [10]:
#used for fixing a certifi problem with the profile. Props to github user marschhuynh for the code, god bless him

import os
import os.path
import ssl
import stat
import subprocess
import sys

STAT_0o775 = ( stat.S_IRUSR | stat.S_IWUSR | stat.S_IXUSR
             | stat.S_IRGRP | stat.S_IWGRP | stat.S_IXGRP
             | stat.S_IROTH |                stat.S_IXOTH )


def main():
    openssl_dir, openssl_cafile = os.path.split(
        ssl.get_default_verify_paths().openssl_cafile)

    print(" -- pip install --upgrade certifi")
    subprocess.check_call([sys.executable,
        "-E", "-s", "-m", "pip", "install", "--upgrade", "certifi"])

    import certifi

    # change working directory to the default SSL directory
    os.chdir(openssl_dir)
    relpath_to_certifi_cafile = os.path.relpath(certifi.where())
    print(" -- removing any existing file or link")
    try:
        os.remove(openssl_cafile)
    except FileNotFoundError:
        pass
    print(" -- creating symlink to certifi certificate bundle")
    os.symlink(relpath_to_certifi_cafile, openssl_cafile)
    print(" -- setting permissions")
    os.chmod(openssl_cafile, STAT_0o775)
    print(" -- update complete")

if __name__ == '__main__':
    main()

 -- pip install --upgrade certifi
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 KB 1.2 MB/s eta 0:00:00
  Attempting uninstall: certifi
    Found existing installation: certifi 2023.5.7
    Uninstalling certifi-2023.5.7:
      Successfully uninstalled certifi-2023.5.7
 -- removing any existing file or link
 -- creating symlink to certifi certificate bundle
 -- setting permissions
 -- update complete


You should consider upgrading via the '/usr/local/bin/python3 -m pip install --upgrade pip' command.


In [17]:
import torch
import torchvision
import torchvision.transforms as transforms

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Set hyperparameters
num_epochs = 45
batch_size = 256
learning_rate = 0.001

# Load the ResNet50 model
model = torchvision.models.resnet50(weights=torchvision.models.resnet.ResNet50_Weights.DEFAULT)

# Freeze the parameters of the model
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Sequential(torch.nn.Linear(model.fc.in_features, 120))

# Set the model to run on the device
model = model.to(device)

# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
checkpoints = "/Users/tonywu/lechacks/extra_files/"

cpu


In [15]:
# Initialize transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(degrees=45),
    transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the ImageNet Object Localization Challenge dataset
train_dataset = torchvision.datasets.ImageFolder(
    root= checkpoints + 'Images',
    transform=transform
)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

try:
  model.load_state_dict(torch.load(checkpoints + "model.pt"));
except:
  pass


In [16]:
def train(network, data, loss_function, op, epochs):
    for epoch in range(epochs):
        counter = 0
        correct = 0
        total = 0
        # Loop over the dataloader
        for inputs, labels in data:

            #calculating loss/training
            network.train()
            inputs = inputs.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)
            # Forward pass
            outputs = model(inputs)
            loss = loss_function(outputs, labels)
            # Backward pass
            op.zero_grad()
            loss.backward()
            op.step()

            #getting accuracy
            network.eval()
            outputs = network(inputs)
            _, predicted = torch.max(outputs.data, 1)
            t = labels.size(0)
            c = (predicted == labels).sum().item()
            counter += 1

            total += t
            correct += c

            if (counter%10 == 0):
                print(f'Set [{epoch+1}, {counter}], Loss: {loss.item():.4f}, Acc: {c/t:.4f}')

        print(f'Average Acc: {correct/total:.4f}')
        torch.save(model.state_dict(), "/content/drive/MyDrive/colab_files/model.pt")


train(model, train_loader, criterion, optimizer, num_epochs)

KeyboardInterrupt: 

In [ ]:
def accuracy(net, dataloader):
    net.to(device)
    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        counter = 0;
        for batch in dataloader:
            images, labels = batch[0].to(device), batch[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            counter += 1;
            if (counter%10 == 0):
                print(f'Set [{counter}], Accuracy: {(predicted == labels).sum().item()/labels.size(0)}')
    return correct/total

torch.save(model.state_dict(), "/content/drive/MyDrive/colab_files/model.pt")

print(f'accuracy: {accuracy(model, train_loader)}')